<a href="https://colab.research.google.com/github/ainfanzon/CRL-IAM-Workshop/blob/main/Exercise_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://drive.google.com/uc?id=1XYr9Tyrz31a5kZdo601xD1QWz_YM8-H3">

### CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.

# Identity and Access Management Workshop.
---
## Scalability and Resiliency.

This section explains the concepts of ranges, range sets, replicas and leaders (a.k.a, leaseholders) which are the building blocks for
[Replication and Rebalancing](https://www.cockroachlabs.com/docs/stable/demo-replication-and-rebalancing).

<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td align="right">
          <img src="https://drive.google.com/uc?id=1roJY0K02x6gDV96uXT24ivR_f2gnf2Pb" width="350" height="345">
      </td>
      <td style="width:5%" align="center">
          &emsp;
      </td>
      <td align="left">
          <img src="https://drive.google.com/uc?id=19C2KDL00TdFwcZQX2epcqzMUi4evEMio" width="525" height="275">
      </td>
  </tr>
</table>

</body>
</html>


| Term | Definition |
| --- | --- |
| Range | Data is sorted in a map of key-value pairs. This keyspace is divided into contiguous chunks called ranges, such that every key is found in one range.|
| Replica | A copy of a range stored on a node. By default, there are three replicas (replication factor) of each range on different nodes.|
| Range Set | Is a collection of ranges. Each set has a leader and a number of replicas based on the replication factor|
|Leader | A range in the range set is elected as the leader. The leader is responsible for managing the replication to other ranges in the set (followers).|


In this section you will:

1. Start a three node cluster.
1. Verify the cluster deployment.
1. Load data and verify replication.

#### 1. Start a three node cluster.

Start with three nodes. For the purpose of this workshop each node will represent a region (Europe, U.S. West and U.S. East)  NOTE: In production, it is recommended that each region has a minimum of three nodes on different availability zones (or servers).

- Copy and paste the following statements to your terminal.

> <code>
cockroach start --insecure --listen-addr=10.0.1.2:26257 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8080 --store=/home/cockroach/data/cr_data_1 --background
cockroach start --insecure --listen-addr=10.0.1.2:26258 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8081 --store=/home/cockroach/data/cr_data_2 --background
cockroach start --insecure --listen-addr=10.0.1.2:26259 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8082 --store=/home/cockroach/data/cr_data_3 --background
cockroach init --insecure --host 10.0.1.2
</code>

You should see a `Cluster successfully initialized` message.

#### 2. Verify the cluster deployment

Verify there are three instances of the `cockroach` process running on different ports.

- On your laptop open a terminal window and connect to the GCP compute engine using ssh.

> `ssh ainfanzon@35.235.99.125`

&emsp;&emsp;NOTE: Need to figure out how to move the public key to the CEs in the ansible script `dply_crdb_infra.yml`.

- List all active `cockroach` processes.

> `pgrep -a cockroach`

&emsp;Each process will be listening on the same IP but different port.

> <code>
--listen-addr=10.0.1.2:26257<br>
--listen-addr=10.0.1.2:26258<br>
--listen-addr=10.0.1.2:26259
</code>

- Open another browser tab to display the cockroach [DB Console](http://35.235.99.125:8080/#/overview/list).

&emsp;&emsp;Note there are NO under-replicated ranges, meaning missing ranges.
  
- Display additional information by connecting to a node using __`psycopg2`__ and the *__VM External IP__* address.

In [ ]:
import psycopg2
import pandas as pd

from IPython.display import IFrame, display, HTML, Markdown

pd.set_option('display.max_colwidth', None)

conn = psycopg2.connect(
        database = 'defaultdb'
      , user = 'root'
      , host = '35.235.99.125'                        # Use the GCP Compute Engine external IP address
      , port = '26257'
      , sslmode = 'disable'
)
cursor = conn.cursor()

- Execute the SQL below to obtain: the List node id, address, server version, up-time, number of ranges, number of leases, server status and membership status.

In [ ]:
cursor.execute("""
SELECT gn.node_id AS "Node ID"
     , gn.advertise_sql_address AS "Advertised Address"
     , gn.build_tag AS "Version"
     , current_timestamp() AT TIME ZONE 'UTC' - gn.started_at AS "Up Time"
     , "ranges" AS "Ranges"
     , leases AS "Leaders"
     , CASE WHEN is_live THEN 'LIVE' ELSE 'DEAD' END AS "status"
     , gl.membership
FROM crdb_internal.gossip_nodes AS gn join crdb_internal.gossip_liveness AS gl USING(node_id)
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.set_index('Node ID', inplace=True)
df_result_set

,Advertised Address,Version,Started At,Up Time,Ranges,Leaders,status,membership
Node ID,,,,,,,,
1,10.0.1.2:26257,v24.2.3,2024-10-22 17:32:13.130050,0 days 21:42:19.161584,58,20,LIVE,active
2,10.0.1.2:26258,v24.2.3,2024-10-22 17:32:13.230691,0 days 21:42:19.060943,58,18,LIVE,active
3,10.0.1.2:26259,v24.2.3,2024-10-22 17:32:13.794942,0 days 21:42:18.496692,58,20,LIVE,active


#### 3. Load data and verify replication.

Follow the steps below to create a database and load the data:

- On your laptop open a second terminal window using ssh (see above).

- On the first terminal change to the **/home/cockroarach/dump** directory and execute the **Python** http server.

> <code>
python -m http.server 3000
</code>

- On the second terminal change to the **/home/cockroarach/sql** directory and execute the **iam.sql** script.

> <code>
cockroach sql --host 10.0.1.2  -u root -d default -f iam.sql --insecure
></code>

- Execute the same query as before to compare the number of ranges and their distribution across the servers.

In [ ]:
cursor.execute("""
SELECT gn.node_id AS "Node ID"
     , gn.advertise_sql_address AS "Advertised Address"
     , gn.build_tag AS "Version"
     , current_timestamp() AT TIME ZONE 'UTC' - gn.started_at AS "Up Time"
     , "ranges" AS "Ranges"
     , leases AS "Leaders"
     , CASE WHEN is_live THEN 'LIVE' ELSE 'DEAD' END AS "status"
     , gl.membership
FROM crdb_internal.gossip_nodes AS gn join crdb_internal.gossip_liveness AS gl USING(node_id)
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.set_index('Node ID', inplace=True)
df_result_set

,Advertised Address,Version,Up Time,Ranges,Leaders,status,membership
Node ID,,,,,,,
1,10.0.1.2:26257,v24.2.3,0 days 21:42:19.161584,84,23,LIVE,active
2,10.0.1.2:26258,v24.2.3,0 days 21:42:19.060943,84,38,LIVE,active
3,10.0.1.2:26259,v24.2.3,0 days 21:42:18.496692,84,23,LIVE,active


## Queries

### Reconnect to the IAM **database**

In [ ]:
conn = psycopg2.connect(
        database = 'iam'
      , user = 'root'
      , host = '35.235.99.125'                        # Use the GCP Compute Engine external IP address
      , port = '26257'
      , sslmode = 'disable'
)
cursor = conn.cursor()

### List 5 users in the IAM database

In [ ]:
try:
    cursor.execute("""
      SELECT
          user_name
        , email
      FROM iam_users
      LIMIT 5
    """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    display(df_result_set.style.hide(axis="index"))
except psycopg2.OperationalError as e:
    cursor.close()
    conn.commit()

user_name,email
lgethin0,kcamber0@google.com.au
spress1,labramamov1@cbc.ca
jhenric2,dbasset2@china.com.cn
acommin3,jjerrolt3@so-net.ne.jp
cpfeiffer4,nmumm4@goo.ne.jp


### Find all roles assigned to a specific user

In [ ]:
cursor.execute("""
SELECT
    user_name AS "User Name"
  , role_name AS "Role"
FROM iam_roles INNER JOIN iam_users USING (role_id)
WHERE user_name = 'acommin3'
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

User Name,Role
acommin3,Auditor


### List all roles in the Keycloak database

In [ ]:
cursor.execute("""
SELECT role_name
FROM iam_roles
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

role_name
Admin/Super Admin
User Manager
Group Manager
Auditor
Helpdesk Support
Developer
External User


### Find all users with a specific role

In [ ]:
cursor.execute("""
SELECT user_name
FROM iam_users
WHERE user_id IN (
  SELECT user_id
  FROM iam_roles
  WHERE role_id = (
    SELECT role_id
    FROM iam_roles
    WHERE role_name = 'Admin/Super Admin'
  )
)
LIMIT 5
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

user_name
lgethin0
spress1
jhenric2
acommin3
cpfeiffer4


### Find all users assigned to a specific group
SELECT username FROM users WHERE id IN (SELECT user_id FROM user_groups WHERE group_id = (SELECT id FROM groups WHERE group_name = 'example_group'));

In [ ]:
cursor.execute("""
SELECT
    user_name AS "User Name"
  , log_timestamp AS "Log Entry Time"
  , activity AS "Activity Description"
FROM iam_users INNER join audit_log using (user_id)
WHERE log_timestamp < current_date() - interval '30 days'
LIMIT 10
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

User Name,Log Entry Time,Activity Description
smaggoriniw,2024-07-16 12:05:58,Aliquam quis turpis eget elit sodales scelerisque. Mauris sit amet eros. Suspendisse accumsan tortor quis turpis. Sed ante. Vivamus tortor. Duis mattis egestas metus. Aenean fermentum. Donec ut mauris eget massa tempor convallis.
bkliner1x,2023-11-16 19:35:59,Mauris lacinia sapien quis libero. Nullam sit amet turpis elementum ligula vehicula consequat. Morbi a ipsum.
bvatcher3m,2024-02-06 23:42:00,"Proin leo odio, porttitor id, consequat in, consequat ut, nulla. Sed accumsan felis."
bbegin41,2024-03-09 21:03:31,"Aenean sit amet justo. Morbi ut odio. Cras mi pede, malesuada in, imperdiet et, commodo vulputate, justo. In blandit ultrices enim. Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Proin interdum mauris non ligula pellentesque ultrices. Phasellus id sapien in sapien iaculis congue. Vivamus metus arcu, adipiscing molestie, hendrerit at, vulputate vitae, nisl."
mrosettini4y,2024-08-29 22:18:49,Cras non velit nec nisi vulputate nonummy.
trothchild6d,2024-09-10 03:23:15,"Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus."
mpigginsej,2023-11-23 22:06:52,"Nam ultrices, libero non mattis pulvinar, nulla pede ullamcorper augue, a suscipit nulla elit ac nulla. Sed vel enim sit amet nunc viverra dapibus. Nulla suscipit ligula in lacus. Curabitur at ipsum ac tellus semper interdum. Mauris ullamcorper purus sit amet nulla. Quisque arcu libero, rutrum ac, lobortis vel, dapibus at, diam. Nam tristique tortor eu pede."
ikuschkego,2024-04-12 08:48:08,"Nunc nisl. Duis bibendum, felis sed interdum venenatis, turpis enim blandit mi, in porttitor pede justo eu massa. Donec dapibus. Duis at velit eu est congue elementum. In hac habitasse platea dictumst. Morbi vestibulum, velit id pretium iaculis, diam erat fermentum justo, nec condimentum neque sapien placerat ante. Nulla justo. Aliquam quis turpis eget elit sodales scelerisque. Mauris sit amet eros. Suspendisse accumsan tortor quis turpis."
psandlandk6,2024-02-08 00:50:52,Proin risus.
bluppittrf,2024-06-28 00:00:57,Etiam faucibus cursus urna. Ut tellus. Nulla ut erat id mauris vulputate elementum. Nullam varius. Nulla facilisi. Cras non velit nec nisi vulputate nonummy. Maecenas tincidunt lacus at velit. Vivamus vel nulla eget eros elementum pellentesque.




-- Find all users assigned to a specific provider:
SELECT email, phone, provider_name
FROM iam_users INNER JOIN auth_provider USING(provider_id)
WHERE provider_type = 'Multi-factor Authentication provider';

-- Find all users assigned to a specific group: SELECT username FROM users WHERE id IN (SELECT user_id FROM user_groups WHERE group_id = (SELECT id FROM groups WHERE group_name = 'example_group'));

-- Count the number of users in Keycloak:
SELECT COUNT(*) FROM iam_users;

-- Find all users who have not logged in for the past 30 days
SELECT user_name, log_timestamp
FROM iam_users INNER join audit_log using (user_id)
WHERE log_timestamp < current_date() - interval '30 days';

---
## CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.
<img src="https://github.com/ainfanzon/Aerospike-Graph/blob/main/source/img/CockroachLabs_Logo.png?raw=1">

---